In [1]:
import openslide as ops
import PIL
import openslide.deepzoom as deepzoom
import large_image
import matplotlib.pyplot as plt
import large_image

In [2]:
def getTileSize(s: ops.OpenSlide):
    """
    Get the tile size.  The tile size isn't in the official openslide
    interface documentation, but every example has the tile size in the
    properties.  If the tile size has an excessive aspect ratio or isn't
    set, fall back to a default of 256 x 256.  The read_region function
    abstracts reading the tiles, so this may be less efficient, but will
    still work.
    """
    # Try to read it, but fall back to 256 if it isn't set.
    width = height = 256
    try:
        width = int(s.properties[
            'openslide.level[0].tile-width'])
    except (ValueError, KeyError):
        pass
    try:
        height = int(s.properties[
            'openslide.level[0].tile-height'])
    except (ValueError, KeyError):
        pass
    # If the tile size is too small (<4) or wrong (<=0), use a default value
    if width < 4:
        width = 256
    if height < 4:
        height = 256
    # If the tile has an excessive aspect ratio, use default values
    if max(width, height) / min(width, height) >= 4:
        width = height = 256
    # Don't let tiles be bigger than the whole image.
    tileWidth = min(width, s.level_dimensions[-1][0])
    tileHeight = min(height, s.level_dimensions[-1][1])
    
    return tileWidth, tileHeight

def getTile(x, y, z, slide: ops.OpenSlide, tileWidth, tileHeight):
    # When we read a region from the SVS, we have to ask for it in the
    # SVS level 0 coordinate system.  Our x and y is in tile space at the
    # specified z level, so the offset in SVS level 0 coordinates has to be
    # scaled by the tile size and by the z level.
    scale = 2 ** (slide.level_count - 1 - z)
    offsetx = x * tileWidth * scale
    offsety = y * tileHeight * scale

    retries = 3
    while retries > 0:
        try:
            tile = slide.read_region(
                (offsetx, offsety), z,
                (tileWidth * scale,
                 tileHeight * scale))
            break
        except ops.lowlevel.OpenSlideError as exc:
            raise exc
    # Always scale to the svs level 0 tile size.
    if scale != 1:
        img = tile.resize((tileWidth, tileHeight), getattr(PIL.Image, 'Resampling', PIL.Image).LANCZOS)
    return img
        
        

In [3]:
# test deepzoom image
full_path = "/home/jackson/code/research/histotools/QuickAnnotator/quickannotator/data/example_project/example_image/example_image.svs"

slide = ops.OpenSlide(full_path)
# dz = deepzoom.DeepZoomGenerator(slide, tile_size=254, overlap=1)

In [4]:
w, h = getTileSize(slide)

In [5]:
w

240

In [6]:
h

240

In [13]:
getTile(0, 0, 4, slide, w, h)

TypeError: can't multiply sequence by non-int of type 'float'